In [19]:
import pandas as pd, numpy as np 
# import csv file as a pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/IndraniMandal/ds-assets/main/assets/messy_covid19_southamerica%20-%20covid19_southamerica.csv')
df.head()

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,NaN,0.0,0,0,0.0
1,2020-01-23,Brazil,NaN,0.0,0,0,0.0
2,2020-01-23,Colombia,NaN,0.0,0,0,0.0
3,2020-02-26,Brazil,NaN,1.0,0,0,1.0
4,2020-02-26,Brazil,NaN,1.0,0,0,1.0


In [20]:
df.isnull().sum()

ObservationDate       0
Country_Region        0
Province_State     5989
Confirmed             1
Deaths                3
Recovered             2
Active               26
dtype: int64

<p>
The Province_State column has the most missing values at 5989 missing entries. The total number of rows is 23150 so therefore 26% of the entries are missing

This column should be dropped, because it is not critical to the information being shared, and there is too much missing data to confidently calculate the rest.

The other columns would likely not be dropped. There are only a small handful of missing entries, and they are important metrics in the data. 

The best way to handle incorrect information would be to see if the information can be converted into the expected data type and keep it. If it cannot be converted, then it should be removed from the data set to avoid bad data.
</p>

In [21]:
df['Confirmed'].fillna(df['Confirmed'].mean(), inplace=True)#replaces NaN's with the mean
df.Confirmed[df.Confirmed.isnull()]#check to make sure there are no NaNs left

C:\Users\blueh\AppData\Local\Temp\ipykernel_29124\988906556.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Confirmed'].fillna(df['Confirmed'].mean(), inplace=True)#replaces NaN's with the mean


Series([], Name: Confirmed, dtype: float64)

In [22]:
df['Deaths'].fillna(0, inplace=True)#replaces NaN's with the median
df.Deaths[df.Deaths.isnull()]

C:\Users\blueh\AppData\Local\Temp\ipykernel_29124\2145108738.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Deaths'].fillna(0, inplace=True)#replaces NaN's with the median


Series([], Name: Deaths, dtype: object)

In [23]:
df['Recovered'] = pd.to_numeric(df['Recovered'], errors='coerce')
v = df.loc[: ,'Recovered':'Active']
v

,Recovered,Active
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
23145,64.0,0.0
23146,4667.0,67.0
23147,2275.0,2039.0
23148,1755.0,208.0


In [24]:

# import the SimpleImputer class
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
clean_df = imputer.fit_transform(v)

In [25]:
clean_df

array([[    0.,     0.],
       [    0.,     0.],
       [    0.,     0.],
       ...,
       [ 2275.,  2039.],
       [ 1755.,   208.],
       [63346.,  9568.]])

In [26]:

clean_df = pd.DataFrame(clean_df, columns= ['Recovered', 'Active'])
clean_df.head()

,Recovered,Active
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0


In [27]:
df.loc[ : ,'ObservationDate': 'Deaths']

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths
0,2020-01-23,Mexico,NaN,0.0,0
1,2020-01-23,Brazil,NaN,0.0,0
2,2020-01-23,Colombia,NaN,0.0,0
3,2020-02-26,Brazil,NaN,1.0,0
4,2020-02-26,Brazil,NaN,1.0,0
...,...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,NaN,64.0,0
23146,2020-09-28,Suriname,NaN,4836.0,102
23147,2020-09-28,Trinidad and Tobago,NaN,4386.0,72
23148,2020-09-28,Uruguay,NaN,2010.0,47


In [28]:
transformed_df = pd.concat([df.loc[ : ,'ObservationDate': 'Deaths'],clean_df ], axis=1)
transformed_df

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,NaN,0.0,0,0.0,0.0
1,2020-01-23,Brazil,NaN,0.0,0,0.0,0.0
2,2020-01-23,Colombia,NaN,0.0,0,0.0,0.0
3,2020-02-26,Brazil,NaN,1.0,0,0.0,1.0
4,2020-02-26,Brazil,NaN,1.0,0,0.0,1.0
...,...,...,...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,NaN,64.0,0,64.0,0.0
23146,2020-09-28,Suriname,NaN,4836.0,102,4667.0,67.0
23147,2020-09-28,Trinidad and Tobago,NaN,4386.0,72,2275.0,2039.0
23148,2020-09-28,Uruguay,NaN,2010.0,47,1755.0,208.0


In [29]:
# import the SimpleImputer class
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
clean_df = imputer.fit_transform(v)

In [30]:
clean_df

array([[    0.,     0.],
       [    0.,     0.],
       [    0.,     0.],
       ...,
       [ 2275.,  2039.],
       [ 1755.,   208.],
       [63346.,  9568.]])

In [31]:
clean_df = pd.DataFrame(clean_df, columns= ['Recovered', 'Active'])
clean_df.head()

,Recovered,Active
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0


In [32]:
df.loc[ : ,'ObservationDate': 'Deaths']

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths
0,2020-01-23,Mexico,NaN,0.0,0
1,2020-01-23,Brazil,NaN,0.0,0
2,2020-01-23,Colombia,NaN,0.0,0
3,2020-02-26,Brazil,NaN,1.0,0
4,2020-02-26,Brazil,NaN,1.0,0
...,...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,NaN,64.0,0
23146,2020-09-28,Suriname,NaN,4836.0,102
23147,2020-09-28,Trinidad and Tobago,NaN,4386.0,72
23148,2020-09-28,Uruguay,NaN,2010.0,47


In [33]:
transformed_df = pd.concat([df.loc[ : ,'ObservationDate': 'Deaths'],clean_df ], axis=1)
transformed_df

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,NaN,0.0,0,0.0,0.0
1,2020-01-23,Brazil,NaN,0.0,0,0.0,0.0
2,2020-01-23,Colombia,NaN,0.0,0,0.0,0.0
3,2020-02-26,Brazil,NaN,1.0,0,0.0,1.0
4,2020-02-26,Brazil,NaN,1.0,0,0.0,1.0
...,...,...,...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,NaN,64.0,0,64.0,0.0
23146,2020-09-28,Suriname,NaN,4836.0,102,4667.0,67.0
23147,2020-09-28,Trinidad and Tobago,NaN,4386.0,72,2275.0,2039.0
23148,2020-09-28,Uruguay,NaN,2010.0,47,1755.0,208.0


In [37]:
df=transformed_df.groupby('Country_Region')['Confirmed'].max()
df.sort_values(ascending=False).head(10)

Country_Region
Brazil                973142.0
Argentina             723132.0
Peru                  367068.0
Chile                 285939.0
Colombia              265576.0
Ecuador               134965.0
Bolivia               134223.0
Mexico                124314.0
Dominican Republic    111666.0
Panama                111277.0
Name: Confirmed, dtype: float64

<p>
Here are the countries that have the top 10 confirmed cases in the data set after cleaning
</p>